In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Load Data

In [ ]:
train_data_loader = data.DataLoader #TODO
test_data_loader = data.DataLoader #TODO

## Neural Network

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_dim: int, latent_dim: int, output_dim: int):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, latent_dim)
        self.act = nn.ReLU()
        self.linear2 = nn.Linear(latent_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = self.act(x)
        x = self.linear2(x)
        return x
    
    def set_optim(self, learning_rate: float, momentum: float):
        self.optim = optim.Adam(self.parameters(), lr=learning_rate, momentum=momentum)

    def get_loss_fn(self):
        return nn.MSELoss()
    
    def train_model(self, epochs: int, data_loader: data.DataLoader):
        self.train()
        loss_fn = self.get_loss_fn()

        for epoch in range(epochs):
            for inputs, labels in data_loader:
                outputs = self(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                self.optim.step()
                self.optim.zero_grad()

            if epoch % 10 == 1:
                print(f"Epoch: {epoch}, loss: {loss.item():.3}")
    
    def evaluate_model(self, data_loader: data.DataLoader):
        self.eval()
        true_predictions, predicitons_amount = 0., 0.

        with torch.no_grad():
            for inputs, labels in data_loader:
                predictions = self(inputs)
                predictions = predictions.squeeze(dim=1)
                true_predictions += (predictions == labels).sum()
                predicitons_amount += labels.shape[0]

            accuracy = 100.0 * true_predictions / predicitons_amount
        
        print(f"Accuracy of the model: {accuracy:4.2f}%")

In [ ]:
input_dim = 64
latent_dim = 256
output_dim = 8
learning_rate = 0.001
momentum = 0.5

In [ ]:
model = NeuralNet(input_dim=input_dim, latent_dim=latent_dim, output_dim=output_dim)
model.set_optim(learning_rate=learning_rate, momentum=momentum)
loss_fn = model.get_loss_fn()

## Training Loop

In [ ]:
model.train_model(epochs=150, data_loader=[])